In [16]:
import os
import sys

import seaborn as sns
import matplotlib.pyplot as plt

import torch.nn as nn
from torch.utils.data import DataLoader, ConcatDataset

from torchvision import transforms

sys.path.append('/app/scripts')

#from unet import get_pretrained_unet, save_model_weights
from models import save_model_weights
from SkinLesionDataset import SkinLesionDataset
from CrossValidationTrainer import CrossValidationTrainer

In [17]:
# Define main paths 
MAIN_FOLDER = '/app'
DATA_FOLDER = os.path.join(MAIN_FOLDER, 'data')

# Paths to the data
TRAIN_IMAGES_PATH_NO_HAIR = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Training_Input_No_Hair')
TRAIN_IMAGES_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Training_Input')
TRAIN_MASKS_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1_Training_GroundTruth')

VAL_IMAGES_PATH_NO_HAIR = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Validation_Input_No_Hair')
VAL_IMAGES_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Validation_Input')
VAL_MASKS_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1_Validation_GroundTruth')

TEST_IMAGES_PATH_NO_HAIR = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Test_Input_No_Hair')
TEST_IMAGES_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1-2_Test_Input')
TEST_MASKS_PATH = os.path.join(DATA_FOLDER, 'ISIC2018_Task1_Test_GroundTruth')


# Best models folder
BEST_MODEL_DIR = os.path.join(MAIN_FOLDER, 'models')


# "/app/models/unet/"

# Training data path
TRAINING_DATA_DIR = os.path.join(MAIN_FOLDER, 'training_data')

# "/app/training_data/unet/training.csv"

In [18]:
# Choose the model architecture
model_name = 'deep_lab'

if model_name == 'unet':
    from models import get_pretrained_unet
    model_fn = get_pretrained_unet
    model_dir = os.path.join(BEST_MODEL_DIR, "unet")
    training_data_output_dir = os.path.join(TRAINING_DATA_DIR, "unet")
elif model_name == 'unet_plus_plus':
    from models import get_pretrained_unet_plus_plus
    model_fn = get_pretrained_unet_plus_plus
    model_dir = os.path.join(BEST_MODEL_DIR, "unet_plus_plus")
    training_data_output_dir = os.path.join(TRAINING_DATA_DIR, "unet_plus_plus")
elif model_name == 'deep_lab':
    from models import get_pretrained_deep_lab
    model_fn = get_pretrained_deep_lab
    model_dir = os.path.join(BEST_MODEL_DIR, "deep_lab")
    training_data_output_dir = os.path.join(TRAINING_DATA_DIR, "deep_lab")

In [19]:
INPUT_SIZE = (256, 256)
BATCH_SIZE = 8
NUM_FOLDS = 5
MAX_EPOCHS = 100
CRITERION = None
mean = [0.485, 0.456, 0.406]  # Mean for each channel (R, G, B)
std = [0.229, 0.224, 0.225]   # Standard deviation for each channel (R, G, B)

# Transforms
transform = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

In [20]:
# Training dataset
train_dataset = SkinLesionDataset(TRAIN_IMAGES_PATH_NO_HAIR, TRAIN_MASKS_PATH, transform)
val_dataset = SkinLesionDataset(VAL_IMAGES_PATH_NO_HAIR, VAL_MASKS_PATH, transform)
train_val_dataset = ConcatDataset([train_dataset, val_dataset])

# Test dataset
test_dataset = SkinLesionDataset(TEST_IMAGES_PATH_NO_HAIR, TEST_MASKS_PATH, transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [21]:
trainer = CrossValidationTrainer(train_val_dataset, model_fn, criterion=CRITERION, num_folds=NUM_FOLDS, batch_size=BATCH_SIZE, num_epochs=MAX_EPOCHS, lr=0.001, patience=10)
results_df, best_model_weights = trainer.cross_validate()

Using device: cuda
FOLD 1/5

Epoch 1/100


Training: 100%|██████████| 270/270 [01:21<00:00,  3.32it/s]


Epoch 1 Training: Loss: 0.2168, IoU: 0.6823


Validating: 100%|██████████| 68/68 [00:20<00:00,  3.32it/s]


Epoch 1 Validation: Loss: 0.1935, IoU: 0.7047
Best model updated!
Learning Rate: 0.001

Epoch 2/100


...

Epoch 23 Validation: Loss: 0.1019, IoU: 0.8284
Learning Rate: 1.0000000000000002e-06

Epoch 24/100


Training: 100%|██████████| 270/270 [01:24<00:00,  3.21it/s]


Epoch 24 Training: Loss: 0.0665, IoU: 0.8797


Validating: 100%|██████████| 68/68 [00:21<00:00,  3.21it/s]

Epoch 24 Validation: Loss: 0.1018, IoU: 0.8285
Learning Rate: 1.0000000000000002e-06
Early stopping at epoch 24 with best validation loss: 0.1000


In [22]:
for fold_idx, model_weights in enumerate(best_model_weights):
    # Create a new model instance
    model = model_fn()

    print(fold_idx)
    # Load the best weights into the model
    model.load_state_dict(model_weights)

    # Save the model weights to a file
    save_model_weights(model, f"{model_dir}/best_segmentation_{fold_idx + 1}.pth")
    results_df.to_csv(os.path.join(training_data_output_dir, 'training.csv'), index=False) 

0
1
2
3
4


In [23]:
training_data_output_dir

'/app/training_data/deep_lab'

In [24]:
for fold_idx, model_weights in enumerate(best_model_weights):

    print(f"{model_dir}/best_segmentation_model_{fold_idx + 1}.pth")

/app/models/deep_lab/best_segmentation_model_1.pth
/app/models/deep_lab/best_segmentation_model_2.pth
/app/models/deep_lab/best_segmentation_model_3.pth
/app/models/deep_lab/best_segmentation_model_4.pth
/app/models/deep_lab/best_segmentation_model_5.pth
